# SK: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
#pip install --upgrade semantic-kernel

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
import semantic_kernel as sk
import os
import logging
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('__name__')
kernel=sk.Kernel(log=logger)

api_key = os.environ['OPENAI_API_KEY']
kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key)
)

In [3]:
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding
kernel.add_text_embedding_generation_service(
        "ada", OpenAITextEmbedding("text-embedding-ada-002", api_key)
    )

In [4]:
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory="catalog"))

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:clickhouse_connect.driver.ctypes:Successfully imported ClickHouse Connect C data optimizations
DEBUG:clickhouse_connect.driver.ctypes:Successfully import ClickHouse Connect C/Numpy optimizations
INFO:clickhouse_connect.json_impl:Using orjson library for writing JSON byte strings
INFO:chromadb.db.duckdb:loaded in 1000 embeddings
INFO:chromadb.db.duckdb:loaded in 1 collections


In [5]:
kernel.import_skill(sk.core_skills.TextMemorySkill())

DEBUG:__name__:Importing skill _GLOBAL_FUNCTIONS_ into the global namespace
DEBUG:__name__:Methods imported: 2


{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7fd18c06c790>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7fd1426902e0>}

In [6]:
query ="Please suggest a shirt with sunblocking"

In [10]:
docs = await kernel.memory.search_async(collection="outdoordb", limit=5, min_relevance_score=0.3, query=query)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"model": "text-embedding-ada-002", "input": ["Please suggest a shirt with sunblocking"], "encoding_format": "base64"}' message='Post details'
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=26 request_id=cc8657e00de013e25f168f2e091b6f60 response_code=200
DEBUG:chromadb.db.index.hnswlib:time to pre process our knn query: 2.86102294921875e-06
DEBUG:chromadb.db.index.hnswlib:time to run knn query: 0.0021409988403320312


In [31]:
async def ragqna(kernel, query, limit) -> str:
    docs = await kernel.memory.search_async(collection="outdoordb", limit=limit, min_relevance_score=0.3, query=query)
    qdocs = "\n```\n".join([docs[i].text for i in range(len(docs))])
    
    prompt = """{{ $qdocs}} 
    
    Question: Please query above documents delimited by triple backticks for {{ $query }} 
    and return results in a table in markdown and summarize each one.
    """
    
    summarize = kernel.create_semantic_function(prompt, temperature=0.0)
    context_variables = sk.ContextVariables(variables={
        "qdocs": qdocs,
        "query": query
    })
    response = summarize(variables=context_variables)
    return response

In [34]:
result = await ragqna(kernel, "shirts with sunblocking",5)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"model": "text-embedding-ada-002", "input": ["shirts with sunblocking"], "encoding_format": "base64"}' message='Post details'
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=22 request_id=4ffa101475614083dfc45bedad6bacbf response_code=200
DEBUG:chromadb.db.index.hnswlib:time to pre process our knn query: 2.384185791015625e-06
DEBUG:chromadb.db.index.hnswlib:time to run knn query: 0.00035119056701660156
DEBUG:__name__:Extracting blocks from template: {{ $qdocs}} 
    
    Question: Please query above documents delimited by triple backticks for {{ $query }} 
    and return results in a table in markdown and summarize each one.
    
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:__name__:Rendering string template: {{ $qdocs}} 
    
    Question: Please query above documents delimited by triple backticks

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=9592 request_id=43575a5dad8acb4f25ff41bfb3f48a73 response_code=200


In [40]:
str(result)


"| Shirt Name | Sun Protection Rating | Fabric Composition | Additional Features |\n| --- | --- | --- | --- |\n| Sun Shield Shirt | UPF 50+ | 78% nylon, 22% Lycra Xtra Life fiber | Moisture-wicking, abrasion-resistant, fits over swimsuit |\n| Men's Plaid Tropic Shirt | UPF 50+ | 52% polyester, 48% nylon | Wrinkle-free, front and back cape venting, two front bellows pockets |\n| Women's Tropical Tee | UPF 50+ | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester | Wrinkle-resistant, low-profile pockets, front and back cape venting, two front pockets, tool tabs, eyewear loop |\n| Men's TropicVibe Shirt | UPF 50+ | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh | Wrinkle-resistant, front and back cape venting, two front bellows pockets |\n| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ | 100% polyester | Wrinkle-resistant, front and back cape venting, two front bellows"

In [42]:
from IPython.display import display, Markdown
display(Markdown(str(result)))

| Shirt Name | Sun Protection Rating | Fabric Composition | Additional Features |
| --- | --- | --- | --- |
| Sun Shield Shirt | UPF 50+ | 78% nylon, 22% Lycra Xtra Life fiber | Moisture-wicking, abrasion-resistant, fits over swimsuit |
| Men's Plaid Tropic Shirt | UPF 50+ | 52% polyester, 48% nylon | Wrinkle-free, front and back cape venting, two front bellows pockets |
| Women's Tropical Tee | UPF 50+ | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester | Wrinkle-resistant, low-profile pockets, front and back cape venting, two front pockets, tool tabs, eyewear loop |
| Men's TropicVibe Shirt | UPF 50+ | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh | Wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ | 100% polyester | Wrinkle-resistant, front and back cape venting, two front bellows